In [1]:
import pandas as pd
import countryinfo as cf # Source: https://gist.github.com/canfixit/1662664

In [2]:
countries = cf.countries
eu_codes = []
eu_names = []
iso_name = {}
name_iso = {}
exclusion_list = ['San Marino', 'Russia', 'Vatican City', 'Monaco']
for co in countries:
    
    # Excluding Russia, Monacco, San Marino and Vatican City bc. lack of data
    if co['name'] in exclusion_list:
        print(co['name'])
        continue
        
    if co['continent'].lower() == 'europe':
        name = co['name']
        if name == 'Kingdom of the Netherlands':
            name = "Netherlands"

        iso = co['code']
        eu_codes.append(iso)
        eu_names.append(name)
        iso_name[iso] = name
        name_iso[name] = iso
        
tot_num_c = len(eu_codes)

print("Number of countries: ", tot_num_c)

Russia
San Marino
Vatican City
Monaco
Number of countries:  40


In [3]:
# Main dataframe
df_main = None
# Time period from 2012 to 2022
time_period = list(range(2011, 2022))

## Temperature Offset
#### - Compared to pre-industrial levels

In [4]:
df_temp_change = pd.read_csv("data/Annual_Surface_Temperature_Change.csv")

df_temp_change = df_temp_change[df_temp_change['ISO2'].isin(eu_codes)]

df_temp_change = df_temp_change.drop(['Unit','Country', 'CTS_Code', 'Indicator', 'Source', 'CTS_Name', 'ObjectId', 'CTS_Full_Descriptor'], axis = 1, inplace=False)
df_temp_change.columns = [str(x).split("F")[-1] for x in df_temp_change.columns]

df_temp_change.columns= df_temp_change.columns.str.lower()
df_temp_change = pd.melt(df_temp_change, id_vars=["iso2", 'iso3'], var_name = ['year'], value_name = 'temp_change C')
df_temp_change["year"] = pd.to_numeric(df_temp_change["year"])

df_temp_change.sort_values(by=["iso3", 'year'], inplace=True)
df_temp_change.columns = ['ISO2', 'ISO3', 'year', 'Temperature Change C']
df_temp_change.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2440 entries, 0 to 2438
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ISO2                  2440 non-null   object 
 1   ISO3                  2440 non-null   object 
 2   year                  2440 non-null   int64  
 3   Temperature Change C  1898 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 95.3+ KB


## CO2 Emissions - Annual 

In [5]:
df_co2 = pd.read_csv("data/annual-co2-emissions-per-country.csv")
df_co2.columns = df_co2.columns.str.lower()
df_co2.columns = ['country', 'ISO3', 'year', 'annual co2 emmisions']
df_co2 = df_co2[df_co2['ISO3'].isin(set(df_temp_change['ISO3']))]
df_co2['y/y Change in Emmision (%)'] = df_co2['annual co2 emmisions'].pct_change()

df_co2["year"] = pd.to_numeric(df_co2["year"])
df_co2 = df_co2[['ISO3', 'year', 'annual co2 emmisions', 'y/y Change in Emmision (%)']]
df_co2.rename(columns={'annual co2 emmisions' : 'annual t co2 emmisions'}, inplace = True)
print("All countries: ", len((set(eu_codes) - set(df_co2['ISO3']))) == 0)
df_co2.info()

All countries:  False
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6556 entries, 517 to 29726
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ISO3                        6556 non-null   object 
 1   year                        6556 non-null   int64  
 2   annual t co2 emmisions      6556 non-null   float64
 3   y/y Change in Emmision (%)  6130 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 256.1+ KB


## GDP

In [6]:
df_gdp = pd.read_csv("data/GDP.csv",delimiter=";")
#Drop last column, as it is no values
df_gdp = df_gdp.iloc[: , :-1]

df_gdp = df_gdp[df_gdp['Country Code'].isin(set(df_temp_change['ISO3']))]
df_gdp.drop(['Country Name', 'Indicator Name', 'Indicator Code'], axis = 1, inplace = True)
df_gdp.columns = df_gdp.columns.str.lower()
df_gdp.rename(columns = {'country code' : "ISO3"}, inplace=True)

df_gdp = pd.melt(df_gdp, id_vars=["ISO3"], var_name = ['year'], value_name = 'GDP US$')
df_gdp["year"] = pd.to_numeric(df_gdp["year"])

print("All countries: ", len((set(eu_codes) - set(df_gdp['ISO3']))) == 0)

df_gdp.info()

All countries:  False
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2480 entries, 0 to 2479
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ISO3     2480 non-null   object 
 1   year     2480 non-null   int64  
 2   GDP US$  1824 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 58.2+ KB


## Green Bonds Annual Issued

In [7]:
# Green Bonds (in Billion $) Probably not inflation adjusted

df_green_bonds = pd.read_csv("data/Green_Bonds.csv")

df_green_bonds = df_green_bonds[df_green_bonds['ISO3'].isin(set(df_temp_change['ISO3']))]

df_green_bonds.drop(['ISO2', 'Indicator', 'Source', 'CTS_Code', 'CTS_Full_Descriptor', 'Type_of_Issuer', 'Use_of_Proceed', 'Principal_Currency'],axis = 1, inplace=True)
df_green_bonds.columns = df_green_bonds.columns.str.lower()
df_green_bonds.columns = [str(x).split("f")[-1] for x in df_green_bonds.columns]

# Only keep cts_name equals Green Bonds Issuances, as we can calculate total later (cts_name == Green Bonds)
df_green_bonds = df_green_bonds.loc[df_green_bonds['cts_name'] == 'Green Bonds Issuances'] 
df_green_bonds.drop(['cts_name', 'unit', 'objectid', 'country'],axis = 1, inplace=True)

df_green_bonds.rename(columns = {'iso3' : "ISO3"}, inplace=True)

df_green_bonds = pd.melt(df_green_bonds, id_vars=["ISO3"], var_name = ['year'], value_name = 'Green Bonds Issuance (Billion US$)')
df_green_bonds["year"] = pd.to_numeric(df_green_bonds["year"])
df_green_bonds.sort_values(by=["ISO3", 'year'], inplace=True)

print("Missing countries: ", (set(eu_codes) - set(df_green_bonds['ISO3'])))

df_green_bonds['Change Green Bonds'] = df_green_bonds['Green Bonds Issuance (Billion US$)'].pct_change()
df_green_bonds

Missing countries:  {'RS', 'AT', 'CH', 'HU', 'DE', 'RO', 'BA', 'ES', 'BE', 'NL', 'AL', 'SE', 'LT', 'LU', 'IS', 'IT', 'BG', 'SI', 'MD', 'GB', 'LV', 'UA', 'PT', 'FR', 'MK', 'FI', 'SK', 'ME', 'GR', 'DK', 'MT', 'IE', 'LI', 'BY', 'CZ', 'NO', 'PL', 'HR', 'EE', 'AD'}


,ISO3,year,Green Bonds Issuance (Billion US$),Change Green Bonds
0,AUT,1985,NaN,NaN
30,AUT,1986,NaN,NaN
60,AUT,1987,NaN,NaN
90,AUT,1990,NaN,NaN
120,AUT,1991,NaN,NaN
...,...,...,...,...
718,UKR,2017,NaN,0.000000
748,UKR,2018,NaN,0.000000
778,UKR,2019,NaN,0.000000
808,UKR,2020,NaN,0.000000


As we observe from the output, we are missing substantial data from some countries, as well as from many years, as green bonds issuance is quite new

## Population

In [8]:
# Population df

df_pop = pd.read_csv("data/population.csv")
df_pop = df_pop[df_pop['Country Code'].isin(set(df_temp_change['ISO3']))]
df_pop.columns = df_pop.columns.str.lower()
df_pop.columns = [str(x).split("[")[0] for x in df_pop.columns]
df_pop.drop(['series name', 'series code', 'country name'] ,axis=1, inplace = True)
df_pop.rename(columns = {'country code' : "ISO3"}, inplace=True)
df_pop = pd.melt(df_pop, id_vars=["ISO3"], var_name = ['year'], value_name = 'population')
df_pop["year"] = pd.to_numeric(df_pop["year"])
df_pop["population"] = pd.to_numeric(df_pop["population"])
df_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ISO3        2000 non-null   object
 1   year        2000 non-null   int64 
 2   population  2000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 47.0+ KB


## Electric Vehicle Sales

BEVs are battery electric vehicles. 

PHEVs are plug-in hybrid electric vehicles. 

FCEVs are fuel cell electric vehicles. 

EVs refers to all electric vehicles (BEVs + PHEVs).

In [9]:
df_sales = pd.read_csv("data/IEA-EV-dataEV salesCarsHistorical.csv")


In [10]:
df_sales[df_sales['region'] == "Norway"].head()


,region,category,parameter,mode,powertrain,year,unit,value
397,Norway,Historical,EV sales,Cars,BEV,2010,sales,360
398,Norway,Historical,EV sales,Cars,BEV,2011,sales,2000
399,Norway,Historical,EV sales,Cars,BEV,2012,sales,3900
400,Norway,Historical,EV sales,Cars,PHEV,2012,sales,320
401,Norway,Historical,EV sales,Cars,PHEV,2013,sales,340


In [11]:

# one for BEVs and one for PHEV ??
df_bev_sales = df_sales[df_sales['powertrain'] == "BEV"]
df_phev_sales = df_sales[df_sales['powertrain'] == "PHEV"]

df_bev_sales = df_bev_sales[['region', 'year', 'value']]
df_phev_sales = df_phev_sales[['region', 'year', 'value']]

s = set(df_phev_sales['region'].unique())
s-set(eu_names)
before = set(eu_names) - s

df_bev_sales = df_bev_sales[df_bev_sales['region'].isin(eu_names)]
df_phev_sales = df_phev_sales[df_phev_sales['region'].isin(eu_names)]

s = set(df_phev_sales['region'].unique())
s-set(eu_names)
after = set(eu_names) - s
print(before - after)
df_bev_sales['region'].replace(name_iso, inplace = True)
df_phev_sales['region'].replace(name_iso, inplace = True)

df_bev_sales.rename(columns = {"region" : "ISO2", "value" : "Battery Electric Vehicles sales"}, inplace = True)
df_phev_sales.rename(columns = {"region" : "ISO2", "value" : "Plugin Hybrid Vehicles sales"}, inplace = True)

df_phev_sales

set()


,ISO2,year,Plugin Hybrid Vehicles sales
23,BE,2011,16
24,BE,2012,260
27,BE,2013,60
28,BE,2014,650
31,BE,2015,2200
...,...,...,...
584,GB,2017,34000
587,GB,2018,35000
588,GB,2019,37000
591,GB,2020,67000


In [12]:
df_ev_chargingP = pd.read_csv("data/IEA-EV-dataEV charging pointsHistorical.csv")
df_ev_chargingP

,region,category,parameter,mode,powertrain,year,unit,value
0,Australia,Historical,EV charging points,EV,Publicly available fast,2017,charging points,40.0
1,Australia,Historical,EV charging points,EV,Publicly available slow,2017,charging points,440.0
2,Australia,Historical,EV charging points,EV,Publicly available fast,2018,charging points,61.0
3,Australia,Historical,EV charging points,EV,Publicly available slow,2018,charging points,670.0
4,Australia,Historical,EV charging points,EV,Publicly available fast,2019,charging points,250.0
...,...,...,...,...,...,...,...,...
522,World,Historical,EV charging points,EV,Publicly available slow,2019,charging points,630000.0
523,World,Historical,EV charging points,EV,Publicly available fast,2020,charging points,390000.0
524,World,Historical,EV charging points,EV,Publicly available slow,2020,charging points,910000.0
525,World,Historical,EV charging points,EV,Publicly available fast,2021,charging points,560000.0


In [13]:
df_ev_chargingP = pd.read_csv("data/IEA-EV-dataEV charging pointsHistorical.csv")

s = set(df_ev_chargingP['region'].unique())
s-set(eu_names)
before = set(eu_names) - s

df_ev_chargingP = df_ev_chargingP[df_ev_chargingP['region'].isin(eu_names)]
df_ev_chargingP = df_ev_chargingP[['region', 'year', 'powertrain', 'value']]
df_ev_chargingP_fast = df_ev_chargingP[df_ev_chargingP['powertrain'] == 'Publicly available fast']
df_ev_chargingP_slow = df_ev_chargingP[df_ev_chargingP['powertrain'] == 'Publicly available slow']

s = set(df_ev_chargingP['region'].unique())
s-set(eu_names)
after = set(eu_names) - s
print(before - after)

df_ev_chargingP = df_ev_chargingP_fast.merge(
    df_ev_chargingP_slow, 
    left_on = ['region', 'year'], 
    right_on = ['region', 'year'],
    how = 'inner',
    suffixes=('_fast', '_slow')
)
df_ev_chargingP['total number of chargingpoints'] = df_ev_chargingP['value_fast'] + df_ev_chargingP['value_slow']
df_ev_chargingP = df_ev_chargingP[['region', 'year', 'value_fast', 'value_slow', 'total number of chargingpoints']]
df_ev_chargingP['region'].replace(name_iso, inplace = True)
df_ev_chargingP.rename(
    columns = {'value_fast': 'number of fast EV charginpoints', 
               'value_slow' : 'number of slow EV charginpoints',
               'region' : 'ISO2'},
                inplace = True)
df_ev_chargingP


set()


,ISO2,year,number of fast EV charginpoints,number of slow EV charginpoints,total number of chargingpoints
0,BE,2013,47.0,330.0,377.0
1,BE,2014,55.0,560.0,615.0
2,BE,2015,77.0,1300.0,1377.0
3,BE,2016,110.0,1500.0,1610.0
4,BE,2017,220.0,1500.0,1720.0
...,...,...,...,...,...
142,GB,2017,2200.0,13000.0,15200.0
143,GB,2018,2700.0,15000.0,17700.0
144,GB,2019,4700.0,22000.0,26700.0
145,GB,2020,6200.0,27000.0,33200.0


# Merging into one main dataframe
- With average and total as two seperate datasheets
- Writing all final DataFrames into a complete Excel file

In [14]:
df_main = df_temp_change

# Check if all conseccutive years are present
# Check if all years still in the dataset for every country

for c in df_main['ISO3'].unique():
    for_this_c = df_main[df_main['ISO3'] == c]
    all_years = (for_this_c['year'] == list(range(min(for_this_c['year']), max(for_this_c['year']) + 1))).all()
    if not all_years:
        print("Not Consecutive years included for: ", c)
        
print("Rest have all years included")

# Check if all countries in dataset
print("Number of countries before: ",len(df_main['ISO2'].unique()) == len(eu_codes))

# Merge instead of join, because join uses index, while merge could specify
# multiple columns
# Using left joint, since we want to hold all values already in the dataframe
# for instance the years, so all missing years in later merged dataframes
# will be set to Nans

# Merge all dataframes

dfs = [df_co2, df_gdp, df_green_bonds, df_pop]

for frame in dfs:
    
    df_main = df_main.merge(frame, left_on=['ISO3','year'], right_on = ['ISO3', 'year'], how = 'left')

# Merge EV-files on ISO2
dfs = [df_phev_sales, df_bev_sales, df_ev_chargingP]
for frame in dfs:
    df_main = df_main.merge(frame, left_on=['ISO2','year'], right_on = ['ISO2', 'year'], how = 'left')
    
# Insert country name
df_main['country'] = df_main['ISO2']
df_main['country'].replace(iso_name, inplace=True)


# Check if all countries still in the dataset
print("Number of countries after: ",len(df_main['country'].unique()) == len(eu_codes))

for c in df_main['ISO3'].unique():
    for_this_c = df_main[df_main['ISO3'] == c]
    all_years = (for_this_c['year'] == list(range(min(for_this_c['year']), max(for_this_c['year']) + 1))).all()
    if not all_years:
        print("Not Consecutive years included for: ", c)
        
print("Rest have all years included")

#f = df_main[df_main['country'] == 'United Kingdom']["annual t co2 emmisions"]
#f
f = df_main[df_main['country'] == 'Germany']["annual t co2 emmisions"]
df_main

Rest have all years included
Number of countries before:  True
Number of countries after:  True
Rest have all years included


,ISO2,ISO3,year,Temperature Change C,annual t co2 emmisions,y/y Change in Emmision (%),GDP US$,Green Bonds Issuance (Billion US$),Change Green Bonds,population,Plugin Hybrid Vehicles sales,Battery Electric Vehicles sales,number of fast EV charginpoints,number of slow EV charginpoints,total number of chargingpoints,country
0,AL,ALB,1961,0.627,2278533.0,0.126742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Albania
1,AL,ALB,1962,0.330,2461733.0,0.080403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Albania
2,AL,ALB,1963,0.068,2080617.0,-0.154816,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Albania
3,AL,ALB,1964,-0.172,2014694.9,-0.031684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Albania
4,AL,ALB,1965,-0.393,2172159.0,0.078158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Albania
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435,UA,UKR,2017,1.401,223087090.0,-0.046671,1.120905e+11,NaN,0.000000,44831135.0,NaN,NaN,NaN,NaN,NaN,Ukraine
2436,UA,UKR,2018,2.222,231674700.0,0.038494,1.308910e+11,NaN,0.000000,44622518.0,NaN,NaN,NaN,NaN,NaN,Ukraine
2437,UA,UKR,2019,2.355,222056700.0,-0.041515,1.538830e+11,NaN,0.000000,44386203.0,NaN,NaN,NaN,NaN,NaN,Ukraine
2438,UA,UKR,2020,3.014,206941170.0,-0.068071,1.566179e+11,NaN,0.000000,44132049.0,NaN,NaN,NaN,NaN,NaN,Ukraine


In [15]:
df_main.describe()

,year,Temperature Change C,annual t co2 emmisions,y/y Change in Emmision (%),GDP US$,Green Bonds Issuance (Billion US$),Change Green Bonds,population,Plugin Hybrid Vehicles sales,Battery Electric Vehicles sales,number of fast EV charginpoints,number of slow EV charginpoints,total number of chargingpoints
count,2440.000000,1898.000000,2.411000e+03,2382.000000,1.808000e+03,158.000000,805.0,2.000000e+03,165.000000,187.000000,147.000000,147.000000,147.000000
mean,1991.000000,0.787887,1.181076e+08,inf,2.979357e+11,5.494571,inf,1.434646e+07,14693.315152,15062.786096,954.789116,8036.979592,8991.768707
std,17.610426,0.872014,1.934903e+08,NaN,6.180695e+11,10.308806,NaN,2.007956e+07,35270.484443,38852.375977,1605.803316,12906.046446,13922.754161
min,1961.000000,-1.793000,0.000000e+00,-0.972617,7.861921e+07,0.000000,-1.0,2.216100e+04,1.000000,2.000000,1.000000,11.000000,22.000000
25%,1976.000000,0.105250,1.356724e+07,-0.026819,1.287717e+10,0.418596,0.0,2.565522e+06,340.000000,395.000000,78.000000,1100.000000,1219.500000
50%,1991.000000,0.824000,4.409910e+07,0.011267,5.911883e+10,1.621302,0.0,6.058844e+06,3100.000000,2200.000000,340.000000,2700.000000,3260.000000
75%,2006.000000,1.445750,1.071690e+08,0.051352,2.498828e+11,5.130566,0.0,1.064820e+07,16000.000000,11000.000000,1000.000000,8300.000000,9260.000000
max,2021.000000,3.595000,1.117888e+09,inf,4.223116e+12,71.700960,inf,8.316087e+07,330000.000000,360000.000000,9200.000000,83000.000000,85600.000000


In [16]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2440 entries, 0 to 2439
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ISO2                                2440 non-null   object 
 1   ISO3                                2440 non-null   object 
 2   year                                2440 non-null   int64  
 3   Temperature Change C                1898 non-null   float64
 4   annual t co2 emmisions              2411 non-null   float64
 5   y/y Change in Emmision (%)          2382 non-null   float64
 6   GDP US$                             1808 non-null   float64
 7   Green Bonds Issuance (Billion US$)  158 non-null    float64
 8   Change Green Bonds                  805 non-null    float64
 9   population                          2000 non-null   float64
 10  Plugin Hybrid Vehicles sales        165 non-null    float64
 11  Battery Electric Vehicles sales     187 non

## Average DataFrame

In [17]:
# Making a average DataFrame
df_average = df_main.groupby(["year"], as_index=False)
df_average = df_average.mean()
df_average


,year,Temperature Change C,annual t co2 emmisions,y/y Change in Emmision (%),GDP US$,Green Bonds Issuance (Billion US$),Change Green Bonds,population,Plugin Hybrid Vehicles sales,Battery Electric Vehicles sales,number of fast EV charginpoints,number of slow EV charginpoints,total number of chargingpoints
0,1961,0.926542,8.194967e+07,0.043623,1.799943e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1962,-0.224583,8.661023e+07,0.069117,1.963547e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1963,-0.520292,9.182950e+07,0.083059,2.166625e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1964,-0.012917,9.604817e+07,0.077956,2.398202e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1965,-0.523917,9.798960e+07,0.040293,2.612632e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2017,1.506300,9.900348e+07,0.022724,4.711856e+11,4.398360,0.700004,1.495601e+07,9490.000,8138.3125,642.4000,7690.133333,8332.533333
57,2018,2.041925,9.717936e+07,-0.007739,5.092313e+11,4.610240,9.078046,1.497802e+07,10917.500,11815.5000,875.0000,8596.000000,9471.000000
58,2019,1.986375,9.294201e+07,-0.028556,5.012861e+11,8.428476,1.822446,1.498761e+07,12311.875,21412.5000,1392.0000,11097.000000,12489.000000
59,2020,2.302875,8.425604e+07,-0.081628,5.001716e+11,8.403601,0.092668,1.499238e+07,37693.750,44086.2500,2087.4375,13908.125000,15995.562500


## Total DataFrame

In [18]:
# Making total DataFrame
df_total = df_main.groupby(["year"], as_index=False)
df_total = df_total.sum()
df_total['y/y Change in Emmision (%)'] = df_total['annual t co2 emmisions'].pct_change()
df_total['Change Green Bonds'] = df_total['Green Bonds Issuance (Billion US$)'].pct_change()
df_total['Change Green Bonds'] = df_total['Change Green Bonds'].astype(float)
df_total['BEV Sales Change'] = df_total['Battery Electric Vehicles sales'].pct_change()
df_total

,year,Temperature Change C,annual t co2 emmisions,y/y Change in Emmision (%),GDP US$,Green Bonds Issuance (Billion US$),Change Green Bonds,population,Plugin Hybrid Vehicles sales,Battery Electric Vehicles sales,number of fast EV charginpoints,number of slow EV charginpoints,total number of chargingpoints,BEV Sales Change
0,1961,22.237,3.196037e+09,NaN,2.879909e+11,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,1962,-5.390,3.377799e+09,0.056871,3.141676e+11,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,1963,-12.487,3.581350e+09,0.060262,3.466600e+11,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,1964,-0.310,3.745879e+09,0.045940,3.837123e+11,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,1965,-12.574,3.821594e+09,0.020213,4.180211e+11,0.000000,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,2017,60.252,3.960139e+09,0.000716,1.884742e+13,65.975395,1.117861,598240534.0,151840.0,130213.0,9636.0,115352.0,124988.0,0.495309
57,2018,81.677,3.887175e+09,-0.018425,2.036925e+13,82.984313,0.257807,599120957.0,174680.0,189048.0,13125.0,128940.0,142065.0,0.451837
58,2019,79.455,3.717680e+09,-0.043603,2.005144e+13,151.712572,0.828208,599504550.0,196990.0,342600.0,22272.0,177552.0,199824.0,0.812238
59,2020,92.115,3.370242e+09,-0.093456,1.950669e+13,168.072022,0.107832,599695156.0,603100.0,705380.0,33399.0,222530.0,255929.0,1.058903


In [19]:
# Only keep values from 2011 to 2021, as these are the most natural for monitoring the impact of paris agreement

df_main = df_main[df_main['year'].isin(set(time_period))]
df_main = df_main.astype({'year' : 'int32'}, copy = True)
df_main.year = pd.to_datetime(df_main.year, format='%Y')

df_average = df_average[df_average['year'].isin(set(time_period))]
df_average = df_average.astype({'year' : 'int32'}, copy = True)
df_average.year = pd.to_datetime(df_average.year, format='%Y')

df_total  = df_total[df_total['year'].isin(set(time_period))]
df_total = df_total.astype({'year' : 'int32'}, copy = True)
df_total.year = pd.to_datetime(df_total.year, format='%Y')

df_main


,ISO2,ISO3,year,Temperature Change C,annual t co2 emmisions,y/y Change in Emmision (%),GDP US$,Green Bonds Issuance (Billion US$),Change Green Bonds,population,Plugin Hybrid Vehicles sales,Battery Electric Vehicles sales,number of fast EV charginpoints,number of slow EV charginpoints,total number of chargingpoints,country
50,AL,ALB,2011-01-01,1.108,5314676.0,0.110959,1.289076e+10,NaN,NaN,2905195.0,NaN,NaN,NaN,NaN,NaN,Albania
51,AL,ALB,2012-01-01,1.568,4850060.0,-0.087421,1.231983e+10,NaN,NaN,2900401.0,NaN,NaN,NaN,NaN,NaN,Albania
52,AL,ALB,2013-01-01,1.444,5287466.0,0.090186,1.277622e+10,NaN,NaN,2895092.0,NaN,NaN,NaN,NaN,NaN,Albania
53,AL,ALB,2014-01-01,1.322,5999658.0,0.134694,1.322815e+10,NaN,NaN,2889104.0,NaN,NaN,NaN,NaN,NaN,Albania
54,AL,ALB,2015-01-01,1.665,4712137.0,-0.214599,1.138685e+10,NaN,NaN,2880703.0,NaN,NaN,NaN,NaN,NaN,Albania
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2435,UA,UKR,2017-01-01,1.401,223087090.0,-0.046671,1.120905e+11,NaN,0.000000,44831135.0,NaN,NaN,NaN,NaN,NaN,Ukraine
2436,UA,UKR,2018-01-01,2.222,231674700.0,0.038494,1.308910e+11,NaN,0.000000,44622518.0,NaN,NaN,NaN,NaN,NaN,Ukraine
2437,UA,UKR,2019-01-01,2.355,222056700.0,-0.041515,1.538830e+11,NaN,0.000000,44386203.0,NaN,NaN,NaN,NaN,NaN,Ukraine
2438,UA,UKR,2020-01-01,3.014,206941170.0,-0.068071,1.566179e+11,NaN,0.000000,44132049.0,NaN,NaN,NaN,NaN,NaN,Ukraine


In [21]:
## Write all DataFrames to excel

writer = pd.ExcelWriter('data/main.xlsx', engine='xlsxwriter')

df_main.to_excel(writer, sheet_name = "main")
df_average.to_excel(writer, sheet_name = "average")
df_total.to_excel(writer, sheet_name = "total")

writer.save()